In [5]:
import requests
import time

# Initialization 
API_KEY = {'X-API-key': 'HCA61YVQ'}
s = requests.Session() 
s.headers.update(API_KEY)

resp = s.get('http://localhost:9999/v1/case')  # removed space before get
resp  # 200 means success

<Response [200]>

In [ ]:
def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        return resp.json()['tick']
    return 0

def get_position(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        for sec in resp.json():
            if sec['ticker'] == ticker:
                return sec['position']
    return 0

def send_order(ticker, action, price, qty):
    params = {
        'ticker': ticker,
        'type': 'LIMIT',
        'quantity': qty,
        'action': action,
        'price': price
    }
    resp = s.post('http://localhost:9999/v1/orders', params=params)
    if resp.ok:
        order_id = resp.json()['order_id']
        print(f'SUCCESS: {action} {qty} {ticker} @ {price}, ID: {order_id}')
        return order_id
    else:
        print(f'FAILED: {resp.text}')
        return None


# # Get all open orders
# resp = s.get('http://localhost:9999/v1/orders')
# if resp.ok:
#     orders = resp.json()
#     print(f"Number of open orders: {len(orders)}")
#     print(f"Orders: {orders}")

def cancel_all():
    resp = s.post('http://localhost:9999/v1/commands/cancel', params={'all': 1})
    if resp.ok:
        result = resp.json()
        print(f'Cancel response: {result}')
        return True
    else:
        print(f'Cancel failed: {resp.text}')
        return False


Orders after cancel: 0 orders
Orders: []


In [ ]:
print('Starting APPL Market Maker\n')

# Post orders
cancel_all()
time.sleep(0.5)  # Wait for cancel to process

send_order('APPL', 'BUY', 0, 1000)
send_order('APPL', 'SELL', 1000, 1000)
print('\nOrders posted! Monitoring...\n')

# Monitor loop
for i in range(180):  # ~15 minutes
    tick = get_tick()
    pos = get_position('APPL')
    
    print(f'Tick: {tick}, Position: {pos}')
    
    if tick > 895:
        print('Case ending soon, stopping...')
        break
    
    time.sleep(5)

# Cleanup
print('\nCleaning up...')
cancel_all()
time.sleep(0.5)
print('Done!')
